# Sentiment Analysis of Donald Trump's Messaging During Key Political Events


#### KEys and Tokens
API_key =
'HFsJLaTSB57fjKEwXyLz9LNmo'

API_key_secret = OMGj9kAIvHw2Sy3VOvlzW98Kca1fX9VNaTiFDC9xBT0HNmyYaW'

access_token =
'1852460704164646912-ACQTSkBUpFgBDy3yCZzJ4IDwEvQsIF'

access_token_secret =
'QFldpaJxAwxgC33ZcfluTYB3sdP69QwVgQTGk4dKBkD3f'


#### Requirements:

In [ ]:
import pandas as pd
import re
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
import tweepy
import time
import zipfile
import ast
import csv
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW, get_scheduler
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt



### Plan:

- Milestone 1 (**November 8**): Collect and preprocess tweets
- Milestone 2 (**November 15**): Train baseline LSTM model and evaluate its performance
- Milestone 3 (**November 22**): Fine-tune the BERT model and finalize the comparison of both models' performance.
- Milestone 4 (**December 8**): Finalize paper

## Data sources:

- Trump Tweets:
  - Trump Tweets: Provides tweets made by Donald Trump, allowing us to study sentiment in his official statements during key political events.

    - https://www.thetrumparchive.com/faq: 80,356 tweets
    - csv: https://drive.google.com/file/d/1xRKHaP-QwACMydlDnyFPEaFdtskJuBa6/view?usp=sharing
      - last updated 01/08/2020 (due to twitter not free)

  - The file realDonaldTrump_in_office.csv contains ALL tweets, including later-deleted tweets, sent by @realDonaldTrump while Donald Trump is in office as the 45th president of the United States (from 20 Jan 2017 to 19 Jan 2021). https://github.com/MarkHershey/CompleteTrumpTweetsArchive

  - The file realDonaldTrump_bf_office.csv contains tweets sent by @realDonaldTrump before Donald Trump took office.

  - Twitter has permanently suspended the account @realDonaldTrump on 08 Jan 2021, so the last tweet captured in this dataset is on 08 Jan 2021.
  - consists of 2 csv files

  - Why: direct way to study public sentiment about a prominent political figure.


- Sentiment140:https://www.kaggle.com/datasets/kazanova/sentiment140
  - Why: pre-labeled for sentiment, making it ideal for benchmarking and comparing the performance of BERT and LSTM models.



here is another data source: https://github.com/chrisalbon/election_day_2016_twitter

# 1. Milestone 1 (Nov 8) Collect and Preprocess


### 1.1 Trump Tweets

1.1.1 BEFORE OFFICE:

processing trump tweets 2009-2021

*  from 2009-05-04
*  to 2021-01-08




In [ ]:
df_trump_up_to_2021 = pd.read_csv('/content/Trump tweets up to_01-08-2021.csv')
df_trump_up_to_2021 = df_trump_up_to_2021.rename(columns={ 'created_at': 'date'})  # Rename columns for uniformi


In [ ]:
df_trump_up_to_2021

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
0,98454970654916608,Republicans and Democrats have both created ou...,f,f,TweetDeck,49,255,2011-08-02 18:07:48,f
1,1234653427789070336,I was thrilled to be back in the Great city of...,f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50,f
2,1218010753434820614,RT @CBS_Herridge: READ: Letter to surveillance...,t,f,Twitter for iPhone,0,7396,2020-01-17 03:22:47,f
3,1304875170860015617,The Unsolicited Mail In Ballot Scam is a major...,f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58,f
4,1218159531554897920,RT @MZHemingway: Very friendly telling of even...,t,f,Twitter for iPhone,0,9081,2020-01-17 13:13:59,f
...,...,...,...,...,...,...,...,...,...
56566,1319485303363571714,RT @RandPaul: I don’t know why @JoeBiden think...,t,f,Twitter for iPhone,0,20683,2020-10-23 03:46:25,f
56567,1319484210101379072,RT @EliseStefanik: President @realDonaldTrump ...,t,f,Twitter for iPhone,0,9869,2020-10-23 03:42:05,f
56568,1319444420861829121,RT @TeamTrump: LIVE: Presidential Debate #Deba...,t,f,Twitter for iPhone,0,8197,2020-10-23 01:03:58,f
56569,1319384118849949702,Just signed an order to support the workers of...,f,f,Twitter for iPhone,176289,36001,2020-10-22 21:04:21,f


In [ ]:
#sort df_trump_up_to_2021 by earliest year. the date column is formatted year-month-date time

# Convert the 'date' column to datetime objects, handling errors
df_trump_up_to_2021['date'] = pd.to_datetime(df_trump_up_to_2021['date'], errors='coerce')

# Sort the DataFrame by the 'date' column in ascending order (earliest first)
df_trump_up_to_2021 = df_trump_up_to_2021.sort_values(by='date')


In [ ]:
df_trump_up_to_2021

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
13325,1698308935,Be sure to tune in and watch Donald Trump on L...,f,f,Twitter Web Client,939,519,2009-05-04 18:54:25,f
13324,1701461182,Donald Trump will be appearing on The View tom...,f,f,Twitter Web Client,259,34,2009-05-05 01:00:10,f
13323,1737479987,Donald Trump reads Top Ten Financial Tips on L...,f,f,Twitter Web Client,37,15,2009-05-08 13:38:08,f
13322,1741160716,New Blog Post: Celebrity Apprentice Finale and...,f,f,Twitter Web Client,29,11,2009-05-08 20:40:15,f
13321,1773561338,"""""""My persona will never be that of a wallflow...",f,f,Twitter Web Client,1877,1321,2009-05-12 14:07:28,f
...,...,...,...,...,...,...,...,...,...
309,1346928882595885058,https://t.co/Pm2PKV0Fp3,f,f,Twitter for iPhone,0,0,2021-01-06 21:17:24,f
311,1346954970910707712,These are the things and events that happen wh...,f,f,Twitter for iPhone,0,0,2021-01-06 23:01:04,f
316,1347334804052844550,https://t.co/csX07ZVWGe,f,f,Twitter Media Studio,629326,154877,2021-01-08 00:10:24,f
323,1347555316863553542,"The 75,000,000 great American Patriots who vot...",f,f,Twitter for iPhone,480997,108844,2021-01-08 14:46:38,f


In [ ]:
df_trump_up_to_2021 = df_trump_up_to_2021.drop(columns=['device'], errors='ignore')

In [ ]:
df_trump_up_to_2021

,id,text,isRetweet,isDeleted,favorites,retweets,date,isFlagged
13325,1698308935,Be sure to tune in and watch Donald Trump on L...,f,f,939,519,2009-05-04 18:54:25,f
13324,1701461182,Donald Trump will be appearing on The View tom...,f,f,259,34,2009-05-05 01:00:10,f
13323,1737479987,Donald Trump reads Top Ten Financial Tips on L...,f,f,37,15,2009-05-08 13:38:08,f
13322,1741160716,New Blog Post: Celebrity Apprentice Finale and...,f,f,29,11,2009-05-08 20:40:15,f
13321,1773561338,"""""""My persona will never be that of a wallflow...",f,f,1877,1321,2009-05-12 14:07:28,f
...,...,...,...,...,...,...,...,...
309,1346928882595885058,https://t.co/Pm2PKV0Fp3,f,f,0,0,2021-01-06 21:17:24,f
311,1346954970910707712,These are the things and events that happen wh...,f,f,0,0,2021-01-06 23:01:04,f
316,1347334804052844550,https://t.co/csX07ZVWGe,f,f,629326,154877,2021-01-08 00:10:24,f
323,1347555316863553542,"The 75,000,000 great American Patriots who vot...",f,f,480997,108844,2021-01-08 14:46:38,f


In [ ]:
# Convert 'f'/'t' to False/True in the 'isRetweet' column
df_trump_up_to_2021['isRetweet'] = df_trump_up_to_2021['isRetweet'].map({'f': False, 't': True})

# Verify the conversion
print(df_trump_up_to_2021['isRetweet'].unique())


[False  True]


In [ ]:
df_trump_up_to_2021['isDeleted'] = df_trump_up_to_2021['isDeleted'].map({'f': False, 't': True})

# Verify the conversion
print(df_trump_up_to_2021['isDeleted'].unique())

[False  True]


In [ ]:
df_trump_up_to_2021['isFlagged'] = df_trump_up_to_2021['isFlagged'].map({'f': False, 't': True})

# Verify the conversion
print(df_trump_up_to_2021['isFlagged'].unique())

[False  True]


In [ ]:
# Save the updated DataFrame to a new CSV file
output_file = 'boolean_corrected_trumpsdf.csv'
df_trump_up_to_2021.to_csv(output_file, index=False)
print(f"Updated DataFrame saved as: {output_file}")

Updated DataFrame saved as: boolean_corrected_trumpsdf.csv


In [ ]:
df_2trump_up_to_2021 = pd.read_csv('/content/boolean_corrected_trumpsdf.csv')

In [ ]:
df_2trump_up_to_2021

,id,text,isRetweet,isDeleted,favorites,retweets,date,isFlagged
0,1698308935,Be sure to tune in and watch Donald Trump on L...,False,False,939,519,2009-05-04 18:54:25,False
1,1701461182,Donald Trump will be appearing on The View tom...,False,False,259,34,2009-05-05 01:00:10,False
2,1737479987,Donald Trump reads Top Ten Financial Tips on L...,False,False,37,15,2009-05-08 13:38:08,False
3,1741160716,New Blog Post: Celebrity Apprentice Finale and...,False,False,29,11,2009-05-08 20:40:15,False
4,1773561338,"""""""My persona will never be that of a wallflow...",False,False,1877,1321,2009-05-12 14:07:28,False
...,...,...,...,...,...,...,...,...
56566,1346928882595885058,https://t.co/Pm2PKV0Fp3,False,False,0,0,2021-01-06 21:17:24,False
56567,1346954970910707712,These are the things and events that happen wh...,False,False,0,0,2021-01-06 23:01:04,False
56568,1347334804052844550,https://t.co/csX07ZVWGe,False,False,629326,154877,2021-01-08 00:10:24,False
56569,1347555316863553542,"The 75,000,000 great American Patriots who vot...",False,False,480997,108844,2021-01-08 14:46:38,False


In [ ]:
df_trump_up_to_2021

,id,text,isRetweet,isDeleted,favorites,retweets,date,isFlagged
13325,1698308935,Be sure to tune in and watch Donald Trump on L...,False,False,939,519,2009-05-04 18:54:25,False
13324,1701461182,Donald Trump will be appearing on The View tom...,False,False,259,34,2009-05-05 01:00:10,False
13323,1737479987,Donald Trump reads Top Ten Financial Tips on L...,False,False,37,15,2009-05-08 13:38:08,False
13322,1741160716,New Blog Post: Celebrity Apprentice Finale and...,False,False,29,11,2009-05-08 20:40:15,False
13321,1773561338,"""""""My persona will never be that of a wallflow...",False,False,1877,1321,2009-05-12 14:07:28,False
...,...,...,...,...,...,...,...,...
309,1346928882595885058,https://t.co/Pm2PKV0Fp3,False,False,0,0,2021-01-06 21:17:24,False
311,1346954970910707712,These are the things and events that happen wh...,False,False,0,0,2021-01-06 23:01:04,False
316,1347334804052844550,https://t.co/csX07ZVWGe,False,False,629326,154877,2021-01-08 00:10:24,False
323,1347555316863553542,"The 75,000,000 great American Patriots who vot...",False,False,480997,108844,2021-01-08 14:46:38,False


In [ ]:
# prompt: find the last date in df_trump_up_to_2021

print(df_trump_up_to_2021['date'].max())

2021-01-08 15:44:28


## 1.1.2 now processing trump tweets after 2021 01 08

In [ ]:

input_file = "/content/realDonaldTrump_before_office.csv"
output_file = "/content/finacleaned_trumptweets.csv"

#handling quoted fields
with open(input_file, mode="r", encoding="utf-8") as infile:
    reader = csv.reader(infile, delimiter=",", quotechar='"')
    rows = list(reader)

#stripping spaces
header = [col.strip() for col in rows[0]]
print("Cleaned Header Row:", header)

tweet_text_start_index = header.index("Tweet Text")

# trke out repeated headers and clean the tweet text
cleaned_rows = []
for row in rows:
    if row != rows[0]:  # here we skip the repeated headers
        # combine the columns after the tweet text column into one column
        if len(row) > len(header):
            row[tweet_text_start_index] = " ".join(row[tweet_text_start_index:]).strip('"')
            row = row[:len(header)]
        cleaned_rows.append(row)

# output cleaned rows to a new file
with open(output_file, mode="w", encoding="utf-8", newline="") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(header)
    writer.writerows(cleaned_rows)

print(f"Cleaned CSV saved to {output_file}")


Cleaned Header Row: ['ID', 'Time', 'Tweet URL', 'Tweet Text']
Cleaned CSV saved to /content/finacleaned_trumptweets.csv


### tweets: 12/31/2020 - 08/01/2022

In [ ]:
import json
import csv

def json_to_csv(json_file_path, csv_file_path):
    """Converts a JSON file to a CSV file.

    Args:
        json_file_path: Path to the input JSON file.
        csv_file_path: Path to the output CSV file.
    """
    try:
        with open(json_file_path, 'r') as json_file:
            data = json.load(json_file)
    except FileNotFoundError:
        print(f"Error: JSON file not found at {json_file_path}")
        return
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in {json_file_path}")
        return

    if not isinstance(data, list):
        print("Error: JSON data must be a list of dictionaries.")
        return

    if not data:  # handle empty JSON list
        print("Warning: Empty JSON data. Creating an empty CSV file.")
        with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
            pass
        return

    # get the headers from the first dictionary in the list
    # incase dict might not have the same keys:
    headers = list(data[0].keys()) if data else []

    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=headers)
        writer.writeheader()
        writer.writerows(data)

json_file = '/content/2021-20208.json'
csv_file = 'output.csv'

json_to_csv(json_file, csv_file)


In [ ]:

file_path = '/content/output.csv'
df = pd.read_csv(file_path)

# convert 'date' column from milliseconds to datetime since need to standardize all the Trump datasets into one at the end!
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'], unit='ms')

# processed dfto a new file
output_file = 'processed_output_with_dates.csv'
df.to_csv(output_file, index=False)
print(f"Processed file saved as: {output_file}")


Processed file saved as: processed_output_with_dates.csv


In [ ]:
df.head(3)

,date,favorites,id,isRetweet,retweets,text
0,2022-08-01 00:02:30.450,18268,108744681091868493,False,5515,"<p>Wow, great dishonesty in politics. Too bad!..."
1,2022-07-31 20:03:26.675,36378,108743741050512893,False,12089,"<p><span class=""quote-inline""><br/>RT: https:/..."
2,2022-07-31 13:28:50.283,56803,108742189401812078,False,17490,"<p>Joe Biden’s second bout of Covid, sometimes..."


In [ ]:
df.columns

Index(['date', 'favorites', 'id', 'isRetweet', 'retweets', 'text'], dtype='object')

## 2022-08-01 - 2022-11-01

In [ ]:
json_file1 = '/content/202208_202211.json'
csv_file1 = 'output1.csv'

json_to_csv(json_file1, csv_file1)

In [ ]:

file_path = '/content/output1.csv'
df2 = pd.read_csv(file_path)

if 'date' in df2.columns:
    df2['date'] = pd.to_datetime(df2['date'], unit='ms')

output_file = 'processed_output_with_dates1.csv'
df2.to_csv(output_file, index=False)
print(f"Processed file saved as: {output_file}")


Processed file saved as: processed_output_with_dates1.csv


In [ ]:
df2.head(3)

,date,favorites,id,isRetweet,retweets,text
0,2022-10-31 22:59:20.689,19849,109265365282926511,False,5881,<p></p>
1,2022-10-31 18:31:45.908,30359,109264313116956131,False,4998,<p>An honor to win the Pro-Am at Doral—LIV Gol...
2,2022-10-31 18:31:09.808,15270,109264310751036449,False,3901,"<p>Crazy that he would get this job, but would..."


## 2022-11-01 to 2023-02-01

In [ ]:
json_file2 = '/content/202211_202302.json'
csv_file2 = 'output2.csv'

json_to_csv(json_file2, csv_file2)

In [ ]:

file_path = '/content/output2.csv'
df3 = pd.read_csv(file_path)

if 'date' in df3.columns:
    df3['date'] = pd.to_datetime(df3['date'], unit='ms')

output_file = 'processed_output_with_dates2.csv'
df3.to_csv(output_file, index=False)
print(f"Processed file saved as: {output_file}")


Processed file saved as: processed_output_with_dates2.csv


# up to 2023-05

In [ ]:
json_file3 = '/content/202305.json'
csv_file3 = 'output3.csv'

json_to_csv(json_file3, csv_file3)

In [ ]:

file_path = '/content/output3.csv'
df4 = pd.read_csv(file_path)

if 'date' in df4.columns:
    df4['date'] = pd.to_datetime(df4['date'], unit='ms')

output_file = 'processed_output_with_dates3.csv'
df4.to_csv(output_file, index=False)
print(f"Processed file saved as: {output_file}")


Processed file saved as: processed_output_with_dates3.csv


# 2023-07-23

In [ ]:
json_file4 = '/content/2023_07_23.json'
csv_file4 = 'output4.csv'

json_to_csv(json_file4, csv_file4)

In [ ]:

file_path = '/content/output4.csv'
df5 = pd.read_csv(file_path)

if 'date' in df5.columns:
    df5['date'] = pd.to_datetime(df5['date'], unit='ms')

output_file = 'processed_output_with_dates4.csv'
df5.to_csv(output_file, index=False)
print(f"Processed file saved as: {output_file}")


Processed file saved as: processed_output_with_dates4.csv


# 2023-09-20

In [ ]:
json_file5 = '/content/2023_09_20.json'
csv_file5 = 'output5.csv'

json_to_csv(json_file5, csv_file5)

In [ ]:

file_path = '/content/output5.csv'
df6 = pd.read_csv(file_path)

if 'date' in df6.columns:
    df6['date'] = pd.to_datetime(df6['date'], unit='ms')

output_file = 'processed_output_with_dates5.csv'
df6.to_csv(output_file, index=False)
print(f"Processed file saved as: {output_file}")


Processed file saved as: processed_output_with_dates5.csv


# 2023-11-25

In [ ]:
json_file6 = '/content/2023_11_25.json'
csv_file6 = 'output6.csv'

json_to_csv(json_file6, csv_file6)

In [ ]:

file_path = '/content/output6.csv'
df7 = pd.read_csv(file_path)

if 'date' in df7.columns:
    df7['date'] = pd.to_datetime(df7['date'], unit='ms')

output_file = 'processed_output_with_dates6.csv'
df7.to_csv(output_file, index=False)
print(f"Processed file saved as: {output_file}")


Processed file saved as: processed_output_with_dates6.csv


# 2024-01-18

In [ ]:
json_file7 = '/content/2024_01_18.json'
csv_file7 = 'output7.csv'

json_to_csv(json_file7, csv_file7)

In [ ]:

file_path = '/content/output7.csv'
df8 = pd.read_csv(file_path)

if 'date' in df8.columns:
    df8['date'] = pd.to_datetime(df8['date'], unit='ms')

output_file = 'processed_output_with_dates7.csv'
df8.to_csv(output_file, index=False)
print(f"Processed file saved as: {output_file}")


Processed file saved as: processed_output_with_dates7.csv


# 2024-03-14

In [ ]:
json_file8 = '/content/2024_03_14.json'
csv_file8 = 'output8.csv'

json_to_csv(json_file8, csv_file8)

In [ ]:
file_path = '/content/output8.csv'
df9 = pd.read_csv(file_path)

if 'date' in df9.columns:
    df9['date'] = pd.to_datetime(df9['date'], unit='ms')

output_file = 'processed_output_with_dates8.csv'
df9.to_csv(output_file, index=False)
print(f"Processed file saved as: {output_file}")

Processed file saved as: processed_output_with_dates8.csv


# 2024_05_20

In [ ]:
json_file9 = '/content/2024_05_20.json'
csv_file9 = 'output9.csv'

json_to_csv(json_file9, csv_file9)

In [ ]:
file_path = '/content/output9.csv'
df10 = pd.read_csv(file_path)

if 'date' in df10.columns:
    df10['date'] = pd.to_datetime(df10['date'], unit='ms')

output_file = 'processed_output_with_dates9.csv'
df10.to_csv(output_file, index=False)
print(f"Processed file saved as: {output_file}")

Processed file saved as: processed_output_with_dates9.csv


# 2024-08-03

In [ ]:
json_file10 = '/content/2024_08_03.json'
csv_file10 = 'output10.csv'

json_to_csv(json_file10, csv_file10)

In [ ]:
file_path = '/content/output10.csv'
df11 = pd.read_csv(file_path)

if 'date' in df11.columns:
    df11['date'] = pd.to_datetime(df11['date'], unit='ms')

output_file = 'processed_output_with_dates10.csv'
df11.to_csv(output_file, index=False)
print(f"Processed file saved as: {output_file}")

Processed file saved as: processed_output_with_dates10.csv


# 2024-09-29

In [ ]:
json_file11 = '/content/2024_09_29.json'
csv_file11 = 'output11.csv'

json_to_csv(json_file11, csv_file11)

In [ ]:
file_path = '/content/output11.csv'
df12 = pd.read_csv(file_path)

if 'date' in df12.columns:
    df12['date'] = pd.to_datetime(df12['date'], unit='ms')

output_file = 'processed_output_with_dates11.csv'
df12.to_csv(output_file, index=False)
print(f"Processed file saved as: {output_file}")

Processed file saved as: processed_output_with_dates11.csv


# 2024_11_24

In [ ]:
json_file12 = '/content/2024_11_24.json'
csv_file12 = 'output12.csv'

json_to_csv(json_file12, csv_file12)

In [ ]:
file_path = '/content/output12.csv'
df13 = pd.read_csv(file_path)

if 'date' in df13.columns:
    df13['date'] = pd.to_datetime(df13['date'], unit='ms')

output_file = 'processed_output_with_dates12.csv'
df13.to_csv(output_file, index=False)
print(f"Processed file saved as: {output_file}")

Processed file saved as: processed_output_with_dates12.csv


## merging datasets

In [ ]:
dfs = [df, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df_trump_up_to_2021]

# Concatenating all Trump dfs
trumpsdf = pd.concat(dfs, ignore_index=True)

# FINAL TRUMP TWEET OUTPUT FILE s
output_file = 'trumpsdf.csv'
trumpsdf.to_csv(output_file, index=False)
print(f"Combined DataFrame saved as: {output_file}")



Combined DataFrame saved as: trumpsdf.csv


### This marks the end of uploading the raw data.
_____________________

---



*Next, we do some further refinement of Trump Tweets:*

# The Trump Tweets output file that encapsulates his tweets from 2009-2024 November is saved as: **'trumpsdf.csv'**

In [ ]:
trumpsdf.head(5)

,date,favorites,id,isRetweet,retweets,text,isDeleted,isFlagged
0,2022-08-01 00:02:30.450,18268,108744681091868493,False,5515,"<p>Wow, great dishonesty in politics. Too bad!...",NaN,NaN
1,2022-07-31 20:03:26.675,36378,108743741050512893,False,12089,"<p><span class=""quote-inline""><br/>RT: https:/...",NaN,NaN
2,2022-07-31 13:28:50.283,56803,108742189401812078,False,17490,"<p>Joe Biden’s second bout of Covid, sometimes...",NaN,NaN
3,2022-07-31 13:08:32.498,21429,108742109593134672,False,4778,<p>Robin Vos is sooo bad for the Great State o...,NaN,NaN
4,2022-07-31 13:04:30.328,19357,108742093722670660,False,5091,"<p><span class=""quote-inline""><br/>RT: https:/...",NaN,NaN


### observing duplicates

to make sure we aren't deleting rows that we need

In [ ]:
# find duplicate rows based on the 'id' column
duplicate_rows = trumpsdf[trumpsdf.duplicated(subset=['id'], keep=False)]


In [ ]:
duplicate_rows

,date,favorites,id,isRetweet,retweets,text,isDeleted,isFlagged
904,2021-01-08 15:44:28,639463,1347569870578266115,False,79113,"To all of those who have asked, I will not be ...",NaN,NaN
905,2021-01-08 14:46:38,535831,1347555316863553542,False,89895,"The 75,000,000 great American Patriots who vot...",NaN,NaN
906,2021-01-08 00:10:24,650194,1347334804052844550,False,130040,https://t.co/csX07ZVWGe,NaN,NaN
907,2021-01-06 23:01:04,0,1346954970910707712,False,0,These are the things and events that happen wh...,NaN,NaN
908,2021-01-06 21:17:24,0,1346928882595885058,False,0,https://t.co/Pm2PKV0Fp3,NaN,NaN
...,...,...,...,...,...,...,...,...
80245,2021-01-06 21:17:24,0,1346928882595885058,False,0,https://t.co/Pm2PKV0Fp3,False,False
80246,2021-01-06 23:01:04,0,1346954970910707712,False,0,These are the things and events that happen wh...,False,False
80247,2021-01-08 00:10:24,629326,1347334804052844550,False,154877,https://t.co/csX07ZVWGe,False,False
80248,2021-01-08 14:46:38,480997,1347555316863553542,False,108844,"The 75,000,000 great American Patriots who vot...",False,False


### Removing duplicates of 'id' while keeping the first occurrence


In [ ]:
trumpsdf = trumpsdf.drop_duplicates(subset=['id'], keep='first')
trumpsdf.reset_index(drop=True, inplace=True)



### Remove isRetweet = True

we do this because for the sentiment analysis we want to only look at Trump's words firsthand.

In [ ]:
trumpsdf = trumpsdf[trumpsdf['isRetweet'] == False]

trumpsdf.reset_index(drop=True, inplace=True)


In [ ]:
trumpsdf

,date,favorites,id,isRetweet,retweets,text,isDeleted,isFlagged
0,2022-08-01 00:02:30.450,18268,108744681091868493,False,5515,"<p>Wow, great dishonesty in politics. Too bad!...",NaN,NaN
1,2022-07-31 20:03:26.675,36378,108743741050512893,False,12089,"<p><span class=""quote-inline""><br/>RT: https:/...",NaN,NaN
2,2022-07-31 13:28:50.283,56803,108742189401812078,False,17490,"<p>Joe Biden’s second bout of Covid, sometimes...",NaN,NaN
3,2022-07-31 13:08:32.498,21429,108742109593134672,False,4778,<p>Robin Vos is sooo bad for the Great State o...,NaN,NaN
4,2022-07-31 13:04:30.328,19357,108742093722670660,False,5091,"<p><span class=""quote-inline""><br/>RT: https:/...",NaN,NaN
...,...,...,...,...,...,...,...,...
65287,2020-12-31 18:52:03.000,237668,1344717975891402759,False,57422,https://t.co/2kYtZF8Mei,False,False
65288,2021-01-01 03:44:48.000,377120,1344852043681435648,False,59151,Finished off the year with the highest Stock M...,False,False
65289,2021-01-01 03:52:33.000,151968,1344853996977197056,False,38293,Sen. Josh Hawley Slams Walmart Tweet Calling H...,False,False
65290,2021-01-01 04:46:02.000,148543,1344867453915029504,False,48343,https://t.co/FDUVk8cm9S,False,False


In [ ]:
# filtered DataFrame to a new CSV file
output_file = 'no_retweets_trumpsdf.csv'
trumpsdf.to_csv(output_file, index=False)
print(f"Filtered DataFrame saved as: {output_file}")

Filtered DataFrame saved as: no_retweets_trumpsdf.csv


### Trump tweets with NO RETWEETS just his direct tweets is saved as: **'no_retweets_trumpsdf.csv'**


# Trump Data Cleaning

### we are only keeping the columns: date, text (tweet), id, favorites (the number of accounts that favorited the tweet), and retweets (the number of times the tweet was shared).

In [ ]:
columns_to_keep = ['date', 'text', 'id', 'favorites', 'retweets']
trumpsdf = trumpsdf[columns_to_keep]


In [ ]:
trumpsdf

,date,text,id,favorites,retweets
0,2022-08-01 00:02:30.450,"<p>Wow, great dishonesty in politics. Too bad!...",108744681091868493,18268,5515
1,2022-07-31 20:03:26.675,"<p><span class=""quote-inline""><br/>RT: https:/...",108743741050512893,36378,12089
2,2022-07-31 13:28:50.283,"<p>Joe Biden’s second bout of Covid, sometimes...",108742189401812078,56803,17490
3,2022-07-31 13:08:32.498,<p>Robin Vos is sooo bad for the Great State o...,108742109593134672,21429,4778
4,2022-07-31 13:04:30.328,"<p><span class=""quote-inline""><br/>RT: https:/...",108742093722670660,19357,5091
...,...,...,...,...,...
65287,2020-12-31 18:52:03.000,https://t.co/2kYtZF8Mei,1344717975891402759,237668,57422
65288,2021-01-01 03:44:48.000,Finished off the year with the highest Stock M...,1344852043681435648,377120,59151
65289,2021-01-01 03:52:33.000,Sen. Josh Hawley Slams Walmart Tweet Calling H...,1344853996977197056,151968,38293
65290,2021-01-01 04:46:02.000,https://t.co/FDUVk8cm9S,1344867453915029504,148543,48343


 dropping missing NAs from 'text' column

In [ ]:
trumpsdf = trumpsdf.dropna(subset=['text'])  # Drop rows with null text

# no rows are removed with this bc no missing text


### using beautiful soup to clean up html in text

In [ ]:
from bs4 import BeautifulSoup

# function to clean html tags
def clean_html(text):
    return BeautifulSoup(text, "html.parser").get_text()

# function to the 'text' column
trumpsdf['text'] = trumpsdf['text'].apply(clean_html)

print(trumpsdf['text'].head())


<ipython-input-200-2fd94b63682d>:5: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  return BeautifulSoup(text, "html.parser").get_text()
<ipython-input-200-2fd94b63682d>:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(text, "html.parser").get_text()


0    Wow, great dishonesty in politics. Too bad! ht...
1    RT: https://truthsocial.com/users/realDonaldTr...
2    Joe Biden’s second bout of Covid, sometimes re...
3    Robin Vos is sooo bad for the Great State of W...
4    RT: https://truthsocial.com/users/realDonaldTr...
Name: text, dtype: object


In [ ]:
trumpsdf.head(4)

,date,text,id,favorites,retweets,cleaned_tweet
0,2022-08-01 00:02:30.450,"Wow, great dishonesty in politics. Too bad! ht...",108744681091868493,18268,5515,pwow great dishonesty in politics too bad a hr...
1,2022-07-31 20:03:26.675,RT: https://truthsocial.com/users/realDonaldTr...,108743741050512893,36378,12089,pspan classquoteinlinebrrt fake news just won...
2,2022-07-31 13:28:50.283,"Joe Biden’s second bout of Covid, sometimes re...",108742189401812078,56803,17490,pjoe bidens second bout of covid sometimes ref...
3,2022-07-31 13:08:32.498,Robin Vos is sooo bad for the Great State of W...,108742109593134672,21429,4778,probin vos is sooo bad for the great state of ...


### Basic cleaning before sentiment analysis

In [ ]:

def clean_text(text):
    text = re.sub(r'http\S+', '', text)  #URLs
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)  # mentions
    text = re.sub(r'#[A-Za-z0-9_]+', '', text)  # hashtags
    text = re.sub(r'[^A-Za-z0-9 ]+', '', text)  # special characters
    text = text.lower()  # to lowercase
    return text


In [ ]:
trumpsdf['text'] = trumpsdf['text'].apply(clean_text)

In [ ]:
trumpsdf.drop(columns=['cleaned_tweet'], inplace=True)

In [ ]:
trumpsdf

,date,text,id,favorites,retweets
0,2022-08-01 00:02:30.450,wow great dishonesty in politics too bad,108744681091868493,18268,5515
1,2022-07-31 20:03:26.675,rt fake news just wont move on this as usual ...,108743741050512893,36378,12089
2,2022-07-31 13:28:50.283,joe bidens second bout of covid sometimes refe...,108742189401812078,56803,17490
3,2022-07-31 13:08:32.498,robin vos is sooo bad for the great state of w...,108742109593134672,21429,4778
4,2022-07-31 13:04:30.328,rt robin vos is only getting worse wants toll...,108742093722670660,19357,5091
...,...,...,...,...,...
65287,2020-12-31 18:52:03.000,,1344717975891402759,237668,57422
65288,2021-01-01 03:44:48.000,finished off the year with the highest stock m...,1344852043681435648,377120,59151
65289,2021-01-01 03:52:33.000,sen josh hawley slams walmart tweet calling hi...,1344853996977197056,151968,38293
65290,2021-01-01 04:46:02.000,,1344867453915029504,148543,48343


We need to remove empty rows in 'text' bc after running beautiful soup to remove any inline html and links we are left with some blanks

In [ ]:
# Remove rows where 'text' is empty or contains only whitespace
trumpsdf = trumpsdf[trumpsdf['text'].str.strip().astype(bool)]

trumpsdf.reset_index(drop=True, inplace=True)


In [ ]:
trumpsdf

,date,text,id,favorites,retweets
0,2022-08-01 00:02:30.450,wow great dishonesty in politics too bad,108744681091868493,18268,5515
1,2022-07-31 20:03:26.675,rt fake news just wont move on this as usual ...,108743741050512893,36378,12089
2,2022-07-31 13:28:50.283,joe bidens second bout of covid sometimes refe...,108742189401812078,56803,17490
3,2022-07-31 13:08:32.498,robin vos is sooo bad for the great state of w...,108742109593134672,21429,4778
4,2022-07-31 13:04:30.328,rt robin vos is only getting worse wants toll...,108742093722670660,19357,5091
...,...,...,...,...,...
55610,2020-12-30 21:27:38.000,watching is almost as bad as watching fake ne...,1344394741031514115,265625,49692
55611,2020-12-30 21:38:11.000,his puppet lt governor and secretary of stat...,1344397397280088070,172187,46133
55612,2020-12-30 21:51:06.000,we now have far more votes than needed to flip...,1344400646066331648,348110,85675
55613,2021-01-01 03:44:48.000,finished off the year with the highest stock m...,1344852043681435648,377120,59151


After final BASIC cleaning, we are left with 55615 rows of Trump Tweets

Advanced clean. Here we would need to use the school's cluster or GPU

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

def advanced_clean_text(text):
    # spelling correction
    #text = str(TextBlob(text).correct())
    # stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    # lemmatization
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

    return text


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
trumpsdf['text'] = trumpsdf['text'].apply(advanced_clean_text)

<ipython-input-222-c253b3854ac4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trumpsdf['text'] = trumpsdf['text'].apply(advanced_clean_text)


In [ ]:
trumpsdf

,date,text,id,favorites,retweets
0,2022-08-01 00:02:30.450,wow great dishonesty politics bad,108744681091868493,18268,5515
1,2022-07-31 20:03:26.675,rt fake news wont move usual fear win,108743741050512893,36378,12089
2,2022-07-31 13:28:50.283,joe bidens second bout covid sometimes referre...,108742189401812078,56803,17490
3,2022-07-31 13:08:32.498,robin vos sooo bad great state wisconsin serio...,108742109593134672,21429,4778
4,2022-07-31 13:04:30.328,rt robin vos getting worse want toll booth put...,108742093722670660,19357,5091
...,...,...,...,...,...
55610,2020-12-30 21:27:38.000,watching almost bad watching fake news new alt...,1344394741031514115,265625,49692
55611,2020-12-30 21:38:11.000,puppet lt governor secretary state disaster ge...,1344397397280088070,172187,46133
55612,2020-12-30 21:51:06.000,far vote needed flip georgia presidential race...,1344400646066331648,348110,85675
55613,2021-01-01 03:44:48.000,finished year highest stock market history set...,1344852043681435648,377120,59151


removing stopwords

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
trumpsdf['text'] = trumpsdf['text'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-225-f5591dbb3dd0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trumpsdf['text'] = trumpsdf['text'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))


In [ ]:
trumpsdf

,date,text,id,favorites,retweets
0,2022-08-01 00:02:30.450,wow great dishonesty politics bad,108744681091868493,18268,5515
1,2022-07-31 20:03:26.675,rt fake news wont move usual fear win,108743741050512893,36378,12089
2,2022-07-31 13:28:50.283,joe bidens second bout covid sometimes referre...,108742189401812078,56803,17490
3,2022-07-31 13:08:32.498,robin vos sooo bad great state wisconsin serio...,108742109593134672,21429,4778
4,2022-07-31 13:04:30.328,rt robin vos getting worse want toll booth put...,108742093722670660,19357,5091
...,...,...,...,...,...
55610,2020-12-30 21:27:38.000,watching almost bad watching fake news new alt...,1344394741031514115,265625,49692
55611,2020-12-30 21:38:11.000,puppet lt governor secretary state disaster ge...,1344397397280088070,172187,46133
55612,2020-12-30 21:51:06.000,far vote needed flip georgia presidential race...,1344400646066331648,348110,85675
55613,2021-01-01 03:44:48.000,finished year highest stock market history set...,1344852043681435648,377120,59151


In [ ]:
# Add the 'source' column with value 'Trump' for all rows
trumpsdf['source'] = 'Trump'

<ipython-input-243-d3daa0ac576e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trumpsdf['source'] = 'Trump'


### Metadata Features
- Engagement Rate:
ratio of (favorites + retweets) to total activity.

- Day of the Week:
Adding the day of the week can help identify trends over time.
python

In [ ]:
trumpsdf['engagement'] = (trumpsdf['favorites'] + trumpsdf['retweets'])

trumpsdf['day_of_week'] = pd.to_datetime(trumpsdf['date']).dt.day_name()

<ipython-input-244-a7a9a71f0372>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trumpsdf['engagement'] = (trumpsdf['favorites'] + trumpsdf['retweets'])
<ipython-input-244-a7a9a71f0372>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trumpsdf['day_of_week'] = pd.to_datetime(trumpsdf['date']).dt.day_name()


In [ ]:
trumpsdf

,date,text,id,favorites,retweets,engagement,day_of_week,source
0,2022-08-01 00:02:30.450,wow great dishonesty politics bad,108744681091868493,18268,5515,23783,Monday,Trump
1,2022-07-31 20:03:26.675,rt fake news wont move usual fear win,108743741050512893,36378,12089,48467,Sunday,Trump
2,2022-07-31 13:28:50.283,joe bidens second bout covid sometimes referre...,108742189401812078,56803,17490,74293,Sunday,Trump
3,2022-07-31 13:08:32.498,robin vos sooo bad great state wisconsin serio...,108742109593134672,21429,4778,26207,Sunday,Trump
4,2022-07-31 13:04:30.328,rt robin vos getting worse want toll booth put...,108742093722670660,19357,5091,24448,Sunday,Trump
...,...,...,...,...,...,...,...,...
55610,2020-12-30 21:27:38.000,watching almost bad watching fake news new alt...,1344394741031514115,265625,49692,315317,Wednesday,Trump
55611,2020-12-30 21:38:11.000,puppet lt governor secretary state disaster ge...,1344397397280088070,172187,46133,218320,Wednesday,Trump
55612,2020-12-30 21:51:06.000,far vote needed flip georgia presidential race...,1344400646066331648,348110,85675,433785,Wednesday,Trump
55613,2021-01-01 03:44:48.000,finished year highest stock market history set...,1344852043681435648,377120,59151,436271,Friday,Trump


In [ ]:
output_file = 'final_cleaned_trump_tweets.csv'
trumpsdf.to_csv(output_file, index=False)

print(f"Cleaned DataFrame saved to: {output_file}")


Cleaned DataFrame saved to: final_cleaned_trump_tweets.csv




---



# Look at outliers if we have time ????????????


1. Handle Outliers:
Outliers can skew the analysis. Consider capping the values at a reasonable threshold, such as the 99th percentile:

2. Check Temporal Patterns:
Engagement can vary significantly over time. Group by date or day_of_week to identify trends:

In [ ]:
print(trumpsdf[['favorites', 'retweets']].describe())


          favorites       retweets
count  5.561500e+04   55615.000000
mean   3.040996e+04    7101.953969
std    5.561701e+04   11856.010112
min    0.000000e+00       0.000000
25%    5.100000e+01      57.000000
50%    7.774000e+03    2375.000000
75%    4.226950e+04   10159.500000
max    1.869706e+06  408866.000000




---



# @ Cole. start running here.

In [ ]:
trumpdf = pd.read_csv('/content/final_cleaned_trump_tweets.csv')


In [ ]:
# Cole's Code
import pandas as pd
trumpdf = pd.read_csv("final_cleaned_trump_tweets.csv")

# Sentiment 140: why>  use as a benchmark to map tweets to positive, negative, or neutral sentiments. EXCEPT we are going to convert to numerical:

- negative sentiment = 0
- neutral sentiment = 2
- positive sentiment = 4

**one thing to remember is that we change the categories since we found out sentiment140 consists of only negative and positive. no neutral.*

In [ ]:
# Cole's Sentiment140 Code

df_sentiment140 = pd.read_csv("sentiment140.csv", encoding='ISO-8859-1', header=None)
df_sentiment140.columns = ['target', 'id', 'date', 'flag', 'user', 'Tweet']
df_sentiment140['source'] = 'Sentiment140'

# Standardize labels
label_mapping = {0: 'negative', 2: 'neutral', 4: 'positive'}
df_sentiment140['sentiment'] = df_sentiment140['target'].map(label_mapping)

#function 'clean_text' already established a couple codes back
df_sentiment140['Tweet'] = df_sentiment140['Tweet'].apply(clean_text)

ParserError: Error tokenizing data. C error: EOF inside string starting at row 540917

#### unzipping and uploading the sentiment140

In [ ]:
zip_path = '/content/sentiment140training.1600000.processed.noemoticon.csv.zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [ ]:
# Load
df_sentiment140 = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1', header=None)
df_sentiment140.columns = ['target', 'id', 'date', 'flag', 'user', 'Tweet']
df_sentiment140['source'] = 'Sentiment140'

# standardizing labels
label_mapping = {0: 'negative', 2: 'neutral', 4: 'positive'}
df_sentiment140['sentiment'] = df_sentiment140['target'].map(label_mapping)


In [ ]:
df_sentiment140.head(5)

,target,id,date,flag,user,Tweet,source,sentiment
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",Sentiment140,negative
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,Sentiment140,negative
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,Sentiment140,negative
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,Sentiment140,negative
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",Sentiment140,negative


keeping only neccessary cols

In [ ]:
df_sentiment140 = df_sentiment140[['Tweet', 'sentiment', 'source']]


In [ ]:
df_sentiment140['Tweet'] = df_sentiment140['Tweet'].apply(clean_text)

NameError: name 'clean_text' is not defined

#### Balance the Dataset
Check if the dataset is balanced across sentiment labels. If not then there is a possibility of undersampling or oversampling to create a balanced dataset.

In [ ]:
print(df_sentiment140['sentiment'].value_counts())

# ex: Undersample to balance the dataset
min_class = df_sentiment140['sentiment'].value_counts().min()
df_sentiment140_balanced = df_sentiment140.groupby('sentiment').apply(lambda x: x.sample(min_class)).reset_index(drop=True)


sentiment
negative    800000
positive    248576
Name: count, dtype: int64


<ipython-input-12-124c774065b8>:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sentiment140_balanced = df_sentiment140.groupby('sentiment').apply(lambda x: x.sample(min_class)).reset_index(drop=True)


# merge the two dfs

In [ ]:
df_trump = trumpsdf[['text', 'source']]  # relevant columns needed for when we merge Trump and Sentiment140 tweets
df_trump.rename(columns={'text': 'Tweet'}, inplace=True)

# Combine
df_combined = pd.concat([df_sentiment140_balanced, df_trump], ignore_index=True)


<ipython-input-250-c5cba5b612ce>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trump.rename(columns={'text': 'Tweet'}, inplace=True)


In [ ]:
df_combined

,Tweet,sentiment,source
0,im so tired but my own cough is keeping me awa...,negative,Sentiment140
1,how tragic its a shame you didnt seek the r...,negative,Sentiment140
2,im with someone else whos making me get rid o...,negative,Sentiment140
3,good morning twitter world ps i miss them,negative,Sentiment140
4,good morning it is monday and i am late ugh,negative,Sentiment140
...,...,...,...
1655610,watching almost bad watching fake news new alt...,NaN,Trump
1655611,puppet lt governor secretary state disaster ge...,NaN,Trump
1655612,far vote needed flip georgia presidential race...,NaN,Trump
1655613,finished year highest stock market history set...,NaN,Trump


# Milestone 2 (Nov 15): LSTM

## gonna have to run this on the computing cluster

In [ ]:
df_sentiment140_balanced

,target,id,date,flag,user,Tweet,source,sentiment
0,0,1694880134,Mon May 04 04:28:11 PDT 2009,NO_QUERY,PhotosByRachel,how long should a girl wait for her man to gro...,Sentiment140,negative
1,0,1965928962,Fri May 29 16:48:15 PDT 2009,NO_QUERY,TomCayman,ecaytrade people who said youd come by stuff a...,Sentiment140,negative
2,0,1881860863,Fri May 22 05:43:39 PDT 2009,NO_QUERY,themichellee,being sick is not fun at all,Sentiment140,negative
3,0,1933010142,Tue May 26 22:45:48 PDT 2009,NO_QUERY,MegLoyal,i cant find a copy of astoriastart again epany...,Sentiment140,negative
4,0,2045287992,Fri Jun 05 10:30:34 PDT 2009,NO_QUERY,kaitlinnelaine,writing some more today trying to add another ...,Sentiment140,negative
...,...,...,...,...,...,...,...,...
497147,4,1760043410,Sun May 10 20:32:00 PDT 2009,NO_QUERY,squeakbot,a twitterand a blogger,Sentiment140,positive
497148,4,1932969157,Tue May 26 22:39:35 PDT 2009,NO_QUERY,SugarLips96,my picture shows up if you follow me i swear ...,Sentiment140,positive
497149,4,1883433263,Fri May 22 08:32:38 PDT 2009,NO_QUERY,sarahgaunt,has had a brilliant day which followed a reall...,Sentiment140,positive
497150,4,1824954282,Sun May 17 03:41:09 PDT 2009,NO_QUERY,Caeleth,gravity amp twittix twibble is for free,Sentiment140,positive


In [ ]:
# Load pre-trained BERT model and tokenizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)
model.to(device)

# pipeline for sent analysis
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

# Apply bert sentiment analysis for the unlabeled data
df_combined['bert_sentiment'] = df_combined.apply(
    lambda row: row['sentiment'] if row['source'] == 'Sentiment140' else sentiment_pipeline(row['Tweet'])[0]['label'], axis=1
)


LSTM Model for Sentiment Analysis

In [ ]:
# tokenize and pad sequences
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df_combined['Tweet'])
sequences = tokenizer.texts_to_sequences(df_combined['Tweet'])
padded_sequences = pad_sequences(sequences, maxlen=100)




In [ ]:
# training and testing sets
train_size = int(0.8 * len(padded_sequences))
train_sequences = padded_sequences[:train_size]
test_sequences = padded_sequences[train_size:]
train_labels = df_combined['bert_sentiment'][:train_size]
test_labels = df_combined['bert_sentiment'][train_size:]

# labels to numerical
label_mapping = {'positive': 2, 'neutral': 1, 'negative': 0}
train_labels = train_labels.map(label_mapping).values
test_labels = test_labels.map(label_mapping).values

# Build LSTM model
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=5000, output_dim=128, input_length=100))
lstm_model.add(LSTM(units=128, return_sequences=False))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(units=3, activation='softmax'))



In [ ]:

lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# train
lstm_model.fit(train_sequences, train_labels, validation_data=(test_sequences, test_labels), epochs=5, batch_size=32)


NameError: name 'lstm_model' is not defined

*this marks the end of milestone 2*

# Milestone 3:(**November 22**): Fine-tune the BERT model and finalize the comparison of both models' performance.

## Fine Tune Bert

In [ ]:

# BERT fine-Tuning
class CustomTweetDataset(Dataset):
    def __init__(self, tweets, labels, tokenizer, max_length):
        self.tweets = tweets
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.tweets)

    def __getitem__(self, index):
        tweet = str(self.tweets[index])
        label = self.labels[index]

        # tokenize and prepare input
        inputs = self.tokenizer(
            tweet,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt",
        )

        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long),
        }

# Prep for fine tuning
train_dataset = CustomTweetDataset(
    df_combined['Tweet'][:train_size],
    train_labels,
    tokenizer,
    max_length=128
)
test_dataset = CustomTweetDataset(
    df_combined['Tweet'][train_size:],
    test_labels,
    tokenizer,
    max_length=128
)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16)

# optimizer and scheduler: Adam
optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = len(train_dataloader) * 3  # 3 epochs
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

# Fine tuning model
model.train()
progress_bar = tqdm(range(num_training_steps))
for epoch in range(3):
    for batch in train_dataloader:
        batch = {key: val.to(device) for key, val in batch.items()}
        outputs = model(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            labels=batch['labels']
        )
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

model.save_pretrained('./fine_tuned_bert_model')
tokenizer.save_pretrained('./fine_tuned_bert_model')

print("BERT model fine-tuned and saved!")


# evaluation bert vs lstm

In [ ]:
#eval Bert
model.eval()
bert_predictions = []
bert_labels = []

for batch in test_dataloader:
    batch = {key: val.to(device) for key, val in batch.items()}
    with torch.no_grad():
        outputs = model(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask']
        )
    predictions = torch.argmax(outputs.logits, dim=-1)
    bert_predictions.extend(predictions.cpu().numpy())
    bert_labels.extend(batch['labels'].cpu().numpy())

# eval LSTM
lstm_predictions = lstm_model.predict(test_sequences)
lstm_predictions = lstm_predictions.argmax(axis=1)

# classification reports
print("LSTM Model Classification Report:")
print(classification_report(test_labels, lstm_predictions, target_names=label_mapping.keys()))

print("Fine-Tuned BERT Model Classification Report:")
print(classification_report(bert_labels, bert_predictions, target_names=label_mapping.keys()))

# confusion matrix
lstm_conf_matrix = confusion_matrix(test_labels, lstm_predictions)
bert_conf_matrix = confusion_matrix(bert_labels, bert_predictions)

# Plot lstm confusion matries
sns.heatmap(lstm_conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=label_mapping.keys(), yticklabels=label_mapping.keys())
plt.title("LSTM Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# Plot bert
sns.heatmap(bert_conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=label_mapping.keys(), yticklabels=label_mapping.keys())
plt.title("BERT Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()
